##Dependencies

In [1]:
try:
    import os
    import time
    import json
    import random
    import nltk
    import discord
    import numpy as np
    import tensorflow as tf
    from googlesearch import search
    from sklearn.tree import DecisionTreeClassifier
    
except ModuleNotFoundError:
    Dependencies = [
        "nltk==3.2.5",
        "numpy==1.19.5",
        "scikit-learn==0.22.2.post1",
        "tensorflow==2.4.1",
        "google==2.0.3",
        "discord"
    ]
    [os.system(f"pip install {Dependency}") for Dependency in Dependencies]
finally:
    os.system("python -m nltk.downloader all")

##Utility Functions

In [2]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if "log_time" in kw:
            name = kw.get("log_name", method.__name__.upper())
            kw["log_time"][name] = int((te - ts) * 1000)
        else:
            print("%r  %2.2f ms" % (method.__name__, (te - ts) * 1000))
        return result

    return timed


def Encode(Text, Binary):
    with open(r"/content/Friday.json", "r") as f:
        data = json.loads(f.read())

    tokens = nltk.RegexpTokenizer(r"\w+").tokenize(Text.lower().strip())
    tags = [tag for word, tag in nltk.pos_tag(tokens)]
    encoded = [data["tags"][tag] for tag in tags]

    if len(encoded) > 51:
        encoded = encoded[:51]
    else:
        for i in range(51 - len(encoded)):
            encoded.append(0)

    if Binary == False:
        return encoded
    elif Binary == True:
        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000)
        encoded = tokenizer.sequences_to_matrix([encoded], mode="binary")
        return encoded[0].tolist()

##Class: Data

In [3]:
class Data:
    def __init__(self):
        pass
 
    # timeit
    def Create(self, BinaryFormat, Run):
        if Run == True:
            with open(r"/content/Friday.json", "r") as f:
                RawData = json.loads(f.read())
 
            TrainJson = RawData["Train"]
            TestJson = RawData["Test"]
 
            X = [Encode(comment, BinaryFormat) for comment in TrainJson]
            Y = [TrainJson[comment] for comment in TrainJson]
            x = [Encode(comment, BinaryFormat) for comment in TestJson]
            y = [TestJson[comment] for comment in TestJson]
 
            FridayData = {"X": X, "Y": Y, "x": x, "y": y}
 
            with open(r"/content/Encoded.json", "w") as f:
                json.dump(FridayData, f, indent=4)
        elif Run == False:
            pass
 
    # timeit
    def Get(self):
        with open(r"/content/Encoded.json", "r") as f:
            EncodedData = json.load(f)
 
        X = EncodedData["X"]
        Y = EncodedData["Y"]
        x = EncodedData["x"]
        y = EncodedData["y"]
 
        Classes = np.max(Y) + 1
 
        Y = tf.keras.utils.to_categorical(Y, Classes)
        y = tf.keras.utils.to_categorical(y, Classes)
 
        MaxWords = 1000
        BatchSize = 256
        Epochs = 10
 
        return [X, Y, x, y, Classes, MaxWords, BatchSize, Epochs]

##Class: Predict

In [4]:
class Predict:
    def __init__(self, Text, Create=False):
        self.Text = Encode(Text, True)
        Data().Create(BinaryFormat=True, Run=Create)
        [
            self.X,
            self.Y,
            self.x,
            self.y,
            self.Classes,
            self.MaxWords,
            self.BatchSize,
            self.Epochs,
        ] = Data().Get()

    # timeit
    def Sklearn(self):

        self.Prediction = (
            DecisionTreeClassifier().fit(self.X, self.Y).predict([self.Text])
        )
        return np.argmax(self.Prediction)

    # timeit
    def Tensorflow(self):
        model = tf.keras.models.Sequential(
            [
                tf.keras.layers.Dense(
                    self.MaxWords, input_shape=(self.MaxWords,), activation="tanh"
                ),
                tf.keras.layers.Dense(2 * self.MaxWords, activation="tanh"),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Dense(3, activation="softmax"),
            ]
        )
        model.compile(
            optimizer="adam", metrics=["accuracy"], loss="categorical_crossentropy"
        )
        model.fit(
            np.array(self.X),
            np.array(self.Y),
            batch_size=self.BatchSize,
            epochs=self.Epochs,
            verbose=0,
        )
        self.Prediction = model.predict([self.Text])
        return np.argmax(self.Prediction)


##Class: Responder

In [5]:
class Response:
    def __init__(self, StrText):
        self.Status = False
        self.Text = StrText
        self.StrText = StrText

    def Native(self, Query):
        with open(r"/content/Friday.json", "r") as F:
            data = json.load(F)

        for _ in data["questions"]:
            if _ in Query.lower():
                for __ in data["questions"][_]:
                    if __ in Query.lower():
                        for ___ in data["questions"][_][__]:
                            if ___ in Query.lower():
                                return [data["questions"][_][__][___], True]
        else:
            return [None, False]

    def Main(self):
        try:
            self.Predictor = Predict(self.Text, False)
        except FileNotFoundError:
            self.Predictor = Predict(self.Text, True)
        
        self.Prediction = self.Predictor.Tensorflow()
        
        with open(r"/content/Friday.json","r") as f:
            data = json.load(f)

        if self.Prediction == 0:
            with open(r"/content/Log.txt", "a") as F:
                F.write(f"\n{self.Text}")
                return "Logged That."

        elif self.Prediction == 1:
            Native_Data = self.Native(Query=self.Text)
            if Native_Data[1] == True:
                self.Status = True
                return Native_Data[0]
            
            elif self.Text.lower() in data["sentences"]["greetings"]:
                return random.choice(data["sentences"]["greetings"])
            
            else:
                return "I can only recognize that this is a Statement, for now."

        elif self.Prediction == 2:
            Native_Data = self.Native(self.Text)
            if Native_Data[1] == True:
                self.Status = True
                return Native_Data[0]
            else:
                for Result in search(self.Text, stop=1):
                    return Result

##Class: Interface

In [7]:
class Interface:
    def __init__(self):
        pass

    def Terminal(self):
        while True:
            Text = input("[Input] : ")
            if Text == "q":
                raise KeyboardInterrupt
            print("[Response] :", Response(Text).Main())

    def Discord(self):
        pass


Interface().Terminal()

[Input] : hi


NameError: ignored

In [ ]:
# import nltk, string
# from sklearn.feature_extraction.text import TfidfVectorizer

# class Similarity():
#     def __init__(self):
#         pass

#     def Normalizer(self,text):
#         tokens = nltk.word_tokenize(text.lower().translate(dict((ord(char), None) for char in string.punctuation)))
#         return [nltk.stem.porter.PorterStemmer().stem(item) for item in tokens]

#     def Check(self,text1, text2):
#         vectorizer = TfidfVectorizer(tokenizer=self.Normalizer, stop_words='english')
#         tfidf = vectorizer.fit_transform([text1, text2])
#         return ((tfidf * tfidf.T).A)[0,1]

# print(Similarity().Check("morning","good morning sweatheart"))